In [ ]:
from IPython.core.display import display, Markdown, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr

In [ ]:
import epics

In [ ]:
from bact_analysis.utils.preprocess import rename_doublicates

In [ ]:
pv_s = epics.PV('Pierre:DT:beam:s')
pv_names = epics.PV('Pierre:DT:beam:names')

In [ ]:
def for_plane(plane):
    assert plane in ["x", "y"]
    pv_nu = epics.PV(f'Pierre:DT:beam:nu:{plane}')
    pv_alpha = epics.PV(f'Pierre:DT:beam:alpha:{plane}')
    pv_beta = epics.PV(f'Pierre:DT:beam:beta:{plane}')
    
    return [pv.get() for pv in (pv_alpha, pv_beta, pv_nu)]
    

In [ ]:
alpha_x, beta_x, mu_x = for_plane("x")
alpha_y, beta_y, mu_y = for_plane("y")


In [ ]:
dmu_x = mu_x[1:] - mu_x[:-1]
dmu_x.shape

In [ ]:
names_twin, s_twin= [pv.get() for pv in (pv_names, pv_s)]


names_twin

In [ ]:
names_twin

In [ ]:
double_used_names, pos_names_without_doublets = rename_doublicates(names_twin.tolist())
double_used_names

In [ ]:
twiss = xr.DataArray(name="twiss", 
                    data=[[alpha_x, alpha_y], [beta_x, beta_y], [mu_x, mu_y]], dims=["par", "plane", "pos"], coords=[["alpha", "beta", "mu"], ["x","y"], pos_names_without_doublets])
mu = xr.DataArray(name="mu", data=[mu_x, mu_y], dims=["plane", "pos"], coords=[["x","y"], pos_names_without_doublets])
mu

In [ ]:
s = xr.DataArray(name="s", data=s_twin, dims=["pos"], coords=[pos_names_without_doublets])

In [ ]:
ttwiss2 = xr.merge([twiss, mu, s])
ttwiss2

In [ ]:
ttwiss = (
    xr.Dataset(dict(beta=ttwiss2.twiss.sel(par="beta")))
    .assign(dict(mu=ttwiss2.twiss.sel(par="mu"), ds=ttwiss2.s))
)

ttwiss = ttwiss.assign(mu=ttwiss.mu).rename(index="pos").assign_coords(dict(pos=pos_names_without_doublets))
ttwiss

In [ ]:
plt.plot(
    ttwiss.ds, ttwiss.mu.sel(plane="y"), 'b-',
    s_twin, mu_y, 'r-',
    
)

In [ ]:
plt.plot(
    ttwiss.ds, ttwiss.beta.sel(par="beta", plane="y"), 'b-',
    s_twin, beta_y, 'r-',
    
)

In [ ]:
ttwiss.to_netcdf('bessyii_twiss_thor_scsi_from_twin.nc')